# Smart Connector for Thermostat API

This notebook acts as a translation layer between the thermostat API and a knowledge engine. It serves as a knowledge base and registers with the knowledge engine using functions `register_post_knowledge_interaction` and `register_react_knowledge_interaction`. In turn, it receives an ID for each interaction that is necessary to post or handle a react.

There are two types of interactions: POST/REACT and ASK/ANSWER. When the graph pattern of this smart connector's post_knowledge_interaction gets matched by the knowledge engine with the graph pattern of another knowledge base's react_knowledge_interaction, the post side sends bindings that are then received and handled by the react side. ASK/ANSWER interactions will be explained in further notebooks.

This smart connector has four main knowledge interactions:

## `(POST)` "post-temp-measurements" 

This knowledge interaction is used to post live temperature measurements to the knowledge engine. It takes the current temperature as measured by the thermostat API, constructs bindings describing the measurement, and sends them to the knowledge engine. This interaction runs in a separate thread in the background.

## `(REACT)`  "set-desired-temp" 

This knowledge interaction changes the desired temperature of the thermostat. When triggered, it calls a function that takes a list of bindings as an argument. This function logs a message indicating that it is changing the desired temperature to the value specified in the binding. Then, it sends a PUT request to the thermostat API's `/desired_temperature` endpoint with the desired temperature value as a query parameter to update the desired temperature of the thermostat. This interaction is handled by the main thread, which is occupied by a handle_loop that handles react KIs.

## `(POST)` "post-humidity-measurements"

This knowledge interaction is used to post live humidity measurements to the knowledge engine. It takes the current humidity as measured by the thermostat API, constructs bindings describing the measurement, and sends them to the knowledge engine. This interaction runs in a separate thread in the background.

## `(REACT)`  "set-desired-humidity"

This knowledge interaction changes the desired humidity of the thermostat. When triggered, it calls a function that takes a list of bindings as an argument. This function logs a message indicating that it is changing the desired humidity to the value specified in the binding. Then, it sends a PUT request to the thermostat API's `/desired_humidity` endpoint with the desired humidity value as a query parameter to update the desired humidity of the thermostat. This interaction is handled by the main thread, which is occupied by a handle_loop that handles react KIs.

All four KIs have their own RDF graph patterns that describe them using the SAREF ontology. These graph patterns are string constants and are hardcoded when registering them with the knowledge engine. After that, only bindings are set and received. These graph patterns can be used by the knowledge engine to interact with this smart connector and other similar knowledge bases.

In [1]:
# ThermostatSC2

In [2]:
import datetime
import logging
import random
import time
import uuid

import helpers as sp
from utils import *

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("ThermostatSC")

This code uses a system called SAREF, which helps different smart devices and applications understand each other and work together. SAREF is like a common language for these devices and applications, and it provides the building blocks to create new ways for them to interact.

The code defines two graph patterns using the Resource Description Framework (RDF), which is a way to represent information about resources on the web. RDF graphs can be visualized as directed graphs with nodes and arcs.

The first graph pattern, `TEMP_MEAS_GRAPH_PATTERN`, describes a temperature measurement. It specifies that the measurement is of type `saref:Measurement`, has a value `?temp`, is measured in `saref:TemperatureUnit`, has a timestamp `?timestamp`, is a measurement of `?room_id`, relates to the property `saref:Temperature`, and is made by `?device_id`.

The second graph pattern, `TEMP_SETTING_GRAPH_PATTERN`, describes the change of the desired temperature setting of the thermostat. It specifies that the setting is of type `saref:SetLevelCommand`, has a value `?desired_temp`, is measured in `saref:TemperatureUnit`, has a timestamp `?timestamp`, is a command of `?room_id`, relates to the property `saref:Temperature`, and is issued by `?device_id`.   

Your task is to create two new graph patterns for humidity, based on the two graph patterns for temperature. The first new graph pattern, `HUMIDITY_MEAS_GRAPH_PATTERN`, should describe a humidity measurement. The second new graph pattern, `HUMIDITY_SETTING_GRAPH_PATTERN`, should describe the change of the desired humidity setting of the thermostat. You can use the existing temperature graph patterns as a guide to create the new humidity graph patterns. Good luck! 😊

In [3]:
THERMOSTAT_API_URL = "http://0.0.0.0:8001/thermostat"
# prefixes of ontologies used in the graph patterns
PREFIXES = {
    "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
    "saref": "https://w3id.org/saref#",
    "xsd": "http://www.w3.org/2001/XMLSchema#",
}
# temp measurement graph pattern
TEMP_MEAS_GRAPH_PATTERN = """?meas rdf:type saref:Measurement .
                        ?meas saref:hasValue ?temp .
                        ?meas saref:isMeasuredIn saref:TemperatureUnit .
                        ?meas saref:hasTimestamp ?timestamp .
                        ?meas saref:isMeasurementOf ?room_id .
                        ?meas saref:relatesToProperty saref:Temperature .
                        ?meas saref:measurementMadeBy ?device_id ."""
# graph pattern describing the change of the ?desired_temp setting of the thermostat
TEMP_SETTING_GRAPH_PATTERN = """?setting rdf:type saref:SetLevelCommand .
                            ?setting saref:hasValue ?desired_temp .
                            ?setting saref:isMeasuredIn saref:TemperatureUnit .
                            ?setting saref:hasTimestamp ?timestamp .
                            ?setting saref:isCommandOf ?room_id .
                            ?setting saref:relatesToProperty saref:Temperature .
                            ?setting saref:commandIssuedBy ?device_id .
                            """
# humidity measurement graph pattern
HUMIDITY_MEAS_GRAPH_PATTERN = """EDIT THIS"""
# graph pattern describing the change of the ?desired_humidity setting of the thermostat
HUMIDITY_SETTING_GRAPH_PATTERN = """EDIT THIS"""

This next cell features a temperature measurement loop that gets triggered every 5 seconds. The function first calls the `get_temp_now` function, which returns the current temperature from the thermostat API. The temperature value is then logged and posted to the knowledge engine using the `post` function.

The data being posted to the knowledge engine is in the form of a graph binding. Each key in the graph binding represents a variable from the graph pattern, denoted by `?variable` (with the question mark prefix). The values for each key are the bindings for the corresponding variable in the graph pattern. For example, `"meas"` is a variable in the graph pattern, and its binding is `f"<{THERMOSTAT_API_URL}/measurements/{str(uuid.uuid4())}>"`. Similarly, `"temp"` is another variable in the graph pattern, and its binding is `f"{value}"`, where `value` is the current temperature returned by the `get_temp_now` function.

All of these values are enclosed in double quotes (`""`) to make them compatible with the knowledge engine. It's important to note that all values being posted to the knowledge engine need to be enclosed in double quotes (`""`) to be compatible with the knowledge engine. For example, the timestamp value is enclosed in double quotes like this: `"timestamp": f'"{sp.get_timestamp_now()}"'`. This ensures that the data can be correctly processed by the knowledge engine.

After posting the temperature measurement, the function logs the published temperature and then waits for 5 seconds using the `time.sleep(5)` command before repeating the loop. This means that a new temperature measurement is taken and posted every 5 seconds.

This graph binding is used to post data to the knowledge engine, where it can be used in conjunction with the temperature measurement graph pattern to make a measurement of temperature.

In [5]:
# DO NOT EDIT
def get_temp_now():
    return requests.get(THERMOSTAT_API_URL + "/actual_temperature").json()

def temp_measurement_loop(post_temp_ki_id, kb_id, ke_endpoint):
    value = get_temp_now()
    logger.info(
        f"attempting to post temp measurement of {value} units at {sp.get_timestamp_now()}"
    )
    post(
        [
            {
                "meas": f"<{THERMOSTAT_API_URL}/measurements/{str(uuid.uuid4())}>",
                "temp": f"{value}",
                "timestamp": f'"{sp.get_timestamp_now()}"',  # ISO 8601 format
                "room_id": f'"{THERMOSTAT_API_URL}/rooms/1"',
                "device_id": f'"{THERMOSTAT_API_URL}/devices/1"',
            }
        ],
        post_temp_ki_id,
        kb_id,
        ke_endpoint,
    )
    logger.info(
        f"Published temperature measurement of {value} units at {sp.get_timestamp_now()}"
    )

    time.sleep(5)

1. Take a look at the temp_measurement_loop function above as an example of how to post data to the knowledge engine.
1. In the humidity_measurement_loop function, replace the {"EDIT": "THIS"} in the post function with a dictionary that contains the current humidity measurement and other relevant information.
1. Use the "meas" and "humidity" keys in the dictionary to represent variables in the graph pattern, and set their bindings accordingly.
1. Add other relevant keys to the dictionary, such as "timestamp", "room_id", and "device_id". Make sure that all values are enclosed in double quotes ("") to make them compatible with the knowledge engine.
1. After posting the humidity measurement, the function logs the published humidity and then waits for 5 seconds using the time.sleep(5) command before repeating the loop.  

This graph binding is used to post data to the knowledge engine, where it can be used in conjunction with the humidity measurement graph pattern to make a measurement of humidity. Remember, you can always refer back to the temp_measurement_loop function as an example of how to structure your code.

In [7]:
def get_humidity_now():
    return requests.get(THERMOSTAT_API_URL + "/actual_humidity").json()

def humidity_measurement_loop(post_humidity_ki_id, kb_id, ke_endpoint):
    value = get_humidity_now()
    logger.info(
        f"attempting to post humidity measurement of {value} units at {sp.get_timestamp_now()}"
    )

    post(
        [{"EDIT": "THIS"}],
        post_humidity_ki_id,
        kb_id,
        ke_endpoint,
    )
    logger.info(
        f"Published humidity measurement of {value} units at {sp.get_timestamp_now()}"
    )

    time.sleep(5)

The `handle_react_change_desired_temp` function is a handler function that is triggered by post knowledge interactions with a matching graph patern, in this case the `TEMP_SETTING_GRAPH_PATTERN`. When triggered, this function sets the thermostat's desired temperature via the REST API.

The function takes in a `bindings` parameter, which contains the bindings of the post knowledge interaction sent from another knowledge base. The function iterates over each binding in the `bindings` list and logs the desired temperature value using the `logger.info` function. The desired temperature value is obtained from the `"desired_temp"` key in the binding dictionary.

After logging the desired temperature value, the function sends a `PUT` request to the thermostat API to change the desired temperature. The desired temperature value is passed as a query parameter in the URL of the `PUT` request.

Once all bindings have been processed, the function returns an empty list. This means that when this handler function is triggered, it will change the thermostat's desired temperature to the value specified in the post knowledge interaction's bindings.

In [8]:
# DO NOT EDIT
def handle_react_change_desired_temp(bindings):
    for binding in bindings:
        logger.info(f"Changing desired temperature to: {binding['desired_temp']}")
        # change desired temp
        requests.put(
            THERMOSTAT_API_URL
            + "/desired_temperature"
            + f'?desired_temperature={int(binding["desired_temp"])}'
        )
    return []

1. Take a look at the handle_react_change_desired_temp function above as an example of how to change the desired value via the REST API.
1. In the handle_react_change_desired_humidity function, replace the # EDIT THIS line with a PUT request to the thermostat API to change the desired humidity.
1. Use the "desired_humidity" key in the binding dictionary to obtain the desired humidity value.
1. Pass the desired humidity value as a query parameter in the URL of the PUT request, similar to how it’s done in the handle_react_change_desired_temp function.
1. Once all bindings have been processed, the function returns an empty list.  

This handler function is triggered by post knowledge interactions with a matching graph binding, in this case a graph pattern for changing the desired humidity. When triggered, it will change the thermostat’s desired humidity to the value specified in the post knowledge interaction’s bindings. 

In [9]:
def handle_react_change_desired_humidity(bindings):
    for binding in bindings:
        logger.info(f"Changing desired humidity to: {binding['desired_humidity']}")
        # change desired temp
        # EDIT THIS
    return []

1. Take a look at the `post_temp_ki_id` and `temp_react_ki_id` lines as examples of how to register post and react knowledge interactions for temperature measurements.
1. Replace the `"EDIT THIS"` line after the `post_temp_ki_id` line with a call to the `register_post_knowledge_interaction` function to register a post knowledge interaction for humidity measurements. Use the appropriate graph pattern, name, and other parameters for the humidity measurement.
1. Replace the `"EDIT THIS"` line after the `temp_react_ki_id` line with a call to the `register_react_knowledge_interaction` function to register a react knowledge interaction for changing the desired humidity. Use the appropriate graph pattern, name, and other parameters for changing the desired humidity.
1. In the `measurement_loop` function, add a call to the `humidity_measurement_loop` function after the call to the `temp_measurement_loop` function. Pass in the appropriate parameters to post humidity measurements to the knowledge engine.
1. In the `start_handle_loop` function, add a key-value pair to the dictionary passed as the first argument. The key should be the `humidity_react_ki_id`, and the value should be the `handle_react_change_desired_humidity` function.

This will allow you to post live humidity measurements to the knowledge engine and handle react knowledge interactions for changing the desired humidity. Remember, you can always refer back to how temperature measurements are handled as an example of how to structure your code, but try to figure out how to make these changes on your own. Good luck! 😊

In [10]:
import threading


def start_sensor_kb(
    kb_id, kb_name, kb_description, ke_endpoint, post_live_measurements=True
):
    # delete in case allready exists
    delete_knowledge_base(kb_id, ke_endpoint)

    # register kb
    register_knowledge_base(kb_id, kb_name, kb_description, ke_endpoint)

    post_temp_ki_id = register_post_knowledge_interaction(
        TEMP_MEAS_GRAPH_PATTERN,
        None,
        "post-temp-measurements",
        kb_id,
        ke_endpoint,
        PREFIXES,
    )

    post_humidity_ki_id = "EDIT THIS"

    temp_react_ki_id = register_react_knowledge_interaction(
        TEMP_SETTING_GRAPH_PATTERN,
        None,
        "set-desired-temp",
        kb_id,
        ke_endpoint,
        PREFIXES,
    )

    humidity_react_ki_id = "EDIT THIS"

    def measurement_loop(post_temp_ki_id, post_humidity_ki_id, kb_id, ke_endpoint):
        while True:
            temp_measurement_loop(post_temp_ki_id, kb_id, ke_endpoint)
            # ADD SOMETHING HERE

    measurement_thread = threading.Thread(
        target=measurement_loop,
        args=(
            post_temp_ki_id,
            post_humidity_ki_id,
            kb_id,
            ke_endpoint,
        ),
    )

    if post_live_measurements:
        measurement_thread.start()

    # Start the start_handle_loop function in a separate thread
    start_handle_loop(
        {
            temp_react_ki_id: handle_react_change_desired_temp,
            # ADD SOMETHING HERE
        },
        kb_id,
        ke_endpoint,
    )

Before running the next cell, make sure that you have completed all the previous steps and edited the code cells as instructed, including implementing the humidity post and handling react to desired humidity.  

Run the code and check the output logs.
If everything is working correctly, you should see log messages indicating that temperature and humidity measurements are being posted to the knowledge engine every 5 seconds.  

You will also see log messages indicating that the desired temperature and humidity are being changed when the `handle_react_change_desired_temp` and `handle_react_change_desired_humidity` functions are triggered by post knowledge interactions with matching graph bindings.  

Note that the log messages indicating that the desired temperature and humidity have been changed (`Changing desired temperature to: 7` and `Changing desired humidity to: X`) are triggered by changing the desired values using the UI smart connector (the next notebook), so you will not see them until you have completed the next notebook.  

It's important to note that a log message like `2023-06-27 14:27:21 INFO {"messageType":"error","message":"Deletion of knowledge base failed, because it could not be found."}` is not an error to be concerned about. This message is generated because the code tries to delete a knowledge base with the same ID before registering it, in order to avoid errors. If the knowledge base with the specified ID does not exist, this log message will be generated, indicating that the deletion of the knowledge base failed because it could not be found. This is expected behavior and is not a cause for concern.

If you see log messages similar to those shown in the example output, then your code is working correctly and you have successfully implemented a system for posting live temperature and humidity measurements to the knowledge engine and handling react knowledge interactions for changing the desired temperature and humidity. Great job! 😊

Example output:

```
2023-06-27 14:27:21 INFO {"messageType":"error","message":"Deletion of knowledge base failed, because it could not be found."}
2023-06-27 14:27:24 INFO registered Thermostat_1
2023-06-27 14:27:24 INFO received issued knowledge interaction id: http://example.org/thermostatSC/interaction/post-temp-measurements
2023-06-27 14:27:24 INFO received issued knowledge interaction id: http://example.org/thermostatSC/interaction/set-desired-temp
2023-06-27 14:27:24 INFO attempting to post temp measurement of 20 units at 2023-06-27T14:27:24+00:00
2023-06-27 14:27:24 INFO Published temperature measurement of 20 units at 2023-06-27T14:27:24+00:00
2023-06-27 14:27:29 INFO attempting to post temp measurement of 22 units at 2023-06-27T14:27:29+00:00
2023-06-27 14:27:29 INFO Published temperature measurement of 22 units at 2023-06-27T14:27:29+00:00
2023-06-27 14:27:34 INFO attempting to post temp measurement of 19 units at 2023-06-27T14:27:34+00:00
2023-06-27 14:27:35 INFO Published temperature measurement of 19 units at 2023-06-27T14:27:35+00:00
2023-06-27 14:27:40 INFO attempting to post temp measurement of 18 units at 2023-06-27T14:27:40+00:00
2023-06-27 14:27:40 INFO Published temperature measurement of 18 units at 2023-06-27T14:27:40+00:00
2023-06-27 14:27:43 INFO Changing desired temperature to: 7
2023-06-27 14:27:45 INFO attempting to post temp measurement of 8 units at 2023-06-27T14:27:45+00:00
2023-06-27 14:27:45 INFO Published temperature measurement of 8 units at 2023-06-27T14:27:45+00:00
2023-06-27 14:27:50 INFO attempting to post temp measurement of 7 units at 2023-06-27T14:27:50+00:00
2023-06-27 14:27:50 INFO Published temperature measurement of 7 units at 2023-06-27T14:27:50+00:00
2023-06-27 14:27:55 INFO attempting to post temp measurement of 6 units at 2023-06-27T14:27:55+00:00
2023-06-27 14:27:55 INFO Published temperature measurement of 6 units at 2023-06-27T14:27:55+00:00
2023-06-27 14:28:00 INFO attempting to post temp measurement of 4 units at 2023-06-27T14:28:00+00:00
2023-06-27 14:28:00 INFO Published temperature measurement of 4 units at 2023-06-27T14:28:00+00:00
2023-06-27 14:28:05 INFO attempting to post temp measurement of 8 units at 2023-06-27T14:28:05+00:00
2023-06-27 14:28:07 INFO Published temperature measurement of 8 units at 2023-06-27T14:28:07+00:00
2023-06-27 14:28:12 INFO attempting to post temp measurement of 9 units at 2023-06-27T14:28:12+00:00
2023-06-27 14:28:13 INFO Published temperature measurement of 9 units at 2023-06-27T14:28:13+00:00
2023-06-27 14:28:18 INFO attempting to post temp measurement of 8 units at 2023-06-27T14:28:18+00:00
2023-06-27 14:28:19 INFO Published temperature measurement of 8 units at 2023-06-27T14:28:19+00:00
2023-06-27 14:28:24 INFO attempting to post temp measurement of 7 units at 2023-06-27T14:28:24+00:00
2023-06-27 14:28:26 INFO Published temperature measurement of 7 units at 2023-06-27T14:28:26+00:00
2023-06-27 14:28:31 INFO attempting to post temp measurement of 7 units at 2023-06-27T14:28:31+00:00
2023-06-27 14:28:32 INFO Published temperature measurement of 7 units at 2023-06-27T14:28:32+00:00
2023-06-27 14:28:37 INFO attempting to post temp measurement of 7 units at 2023-06-27T14:28:37+00:00
2023-06-27 14:28:38 INFO Published temperature measurement of 7 units at 2023-06-27T14:28:38+00:00
2023-06-27 14:28:43 INFO attempting to post temp measurement of 6 units at 2023-06-27T14:28:43+00:00


In [11]:
start_sensor_kb(
    "http://example.org/thermostatSC",
    "Thermostat_1",
    "A thermostat",
    "http://knowledge_engine:8280/rest/",
)

2023-06-27 14:27:21 INFO {"messageType":"error","message":"Deletion of knowledge base failed, because it could not be found."}
2023-06-27 14:27:24 INFO registered Thermostat_1
2023-06-27 14:27:24 INFO received issued knowledge interaction id: http://example.org/thermostatSC/interaction/post-temp-measurements
2023-06-27 14:27:24 INFO received issued knowledge interaction id: http://example.org/thermostatSC/interaction/set-desired-temp
2023-06-27 14:27:24 INFO attempting to post temp measurement of 20 units at 2023-06-27T14:27:24+00:00
2023-06-27 14:27:24 INFO Published temperature measurement of 20 units at 2023-06-27T14:27:24+00:00
2023-06-27 14:27:29 INFO attempting to post temp measurement of 22 units at 2023-06-27T14:27:29+00:00
2023-06-27 14:27:29 INFO Published temperature measurement of 22 units at 2023-06-27T14:27:29+00:00
2023-06-27 14:27:34 INFO attempting to post temp measurement of 19 units at 2023-06-27T14:27:34+00:00
2023-06-27 14:27:35 INFO Published temperature measureme

KeyboardInterrupt: 